In [84]:
import pandas as pd
import pickle
import configparser

In [70]:
train_data = pd.read_csv('../data/raw/train.csv')
test_data = pd.read_csv('../data/raw/test.csv')

In [71]:
with open('../artifacts/pipeline.pkl', 'rb') as f:
    titanic_survived_model = pickle.load(f)

In [85]:
config = configparser.ConfigParser()
config.read('../pipeline.cfg')

['../pipeline.cfg']

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Agregar lectura de target con el archivo de configuraciones

In [ ]:
x_features = train_data.drop(labels=['Survived'], axis=1)
y_target = train_data['Survived']

x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size=0.15, shuffle=True, random_state=2025)

Configuramos y entrenamos pipeline con el modelo.

In [86]:
drop_vars = list(config.get('GENERAL', 'VARS_TO_DROP').split(', '))

In [93]:
x_features_train = titanic_survived_model.fit_transform(x_train)
col_names = list(set(x_train.columns).difference(set(drop_vars)))
df_features_train = pd.DataFrame(x_features_train, columns=col_names)
df_features_train.head()

,Age,Pclass,Parch,SibSp,Embarked,Fare,Sex
0,-1.590253,-4.956738e-01,-0.466123,0.741702,0.612049,-1.503068,0.740309
1,0.822215,4.361245e-01,-0.466123,-0.489588,-0.477489,0.606083,0.740309
2,-1.590253,-1.074245e-01,-0.466123,-0.489588,0.291771,0.606083,0.740309
3,-1.590253,-2.627242e-01,-0.466123,-0.489588,0.917225,0.606083,-1.350787
4,0.822215,-2.758677e-16,-0.466123,-0.489588,-0.482906,0.606083,0.740309


In [95]:
x_features_test = titanic_survived_model.transform(x_test)
col_names = list(set(x_train.columns).difference(set(drop_vars)))
df_features_test = pd.DataFrame(x_features_test, columns=col_names)
df_features_test.head()

,Age,Pclass,Parch,SibSp,Embarked,Fare,Sex
0,0.822215,-7.286234e-01,0.398718,0.741702,-0.325967,-1.503068,0.740309
1,0.822215,6.690740e-01,0.398718,5.666861,-0.017755,0.606083,-1.350787
2,-1.590253,-9.615729e-01,0.398718,-0.489588,1.509191,-1.503068,-1.350787
3,0.822215,-2.758677e-16,0.398718,1.972992,-0.174119,0.606083,0.740309
4,0.822215,-8.839231e-01,-0.466123,-0.489588,-0.472564,0.606083,0.740309


In [97]:
rf_model = RandomForestClassifier().fit(df_features_processed, y_train)
y_preds = rf_model.predict(df_features_test)
rf_acc = accuracy_score(y_preds, y_test)
rf_acc

0.753731343283582

In [98]:
lr_model = LogisticRegression().fit(df_features_processed, y_train)
y_preds_lr = lr_model.predict(df_features_test)
lr_acc = accuracy_score(y_preds_lr, y_test)
lr_acc

0.746268656716418

In [13]:
if(rf_acc > lr_acc):
    titanic_survived_model.steps.append(
        ('modelo_random_forest', RandomForestClassifier()))
else:
    titanic_survived_model.steps.append(
        ('modelo_regresion_logistica', LogisticRegression()))

In [14]:
titanic_survived_model

Pipeline(steps=[('continues_var_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Age', 'Fare'])),
                ('categorical_var_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables='Embarked')),
                ('categorical_encode_ohe',
                 OneHotEncoder(drop_last=True, variables='Sex')),
                ('categorical_encode_frequency',
                 CountFrequencyEncoder(variables='Embarked')),
                ('fueature_scaling', StandardScaler()),
                ('modelo_regresion_logistica', LogisticRegression())])

Entrenamiento de modelo.

In [ ]:
titanic_survived_model.fit()

In [ ]:
dataset_test = pd.read_csv('../data/raw/test.csv')
dataset_test.drop(labels=['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
titanic_survived_model.predict(dataset_test)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.